# Lab 8: Evolutionary computation

### Consider the following example:

Determine the minimum of the function $f(x)= x_1^2+...+x_n^2$ with $x_i \in [-5.12, 5.12]$, $i \in \overline{(1, n)}$

We have an example of steady state genetic algorithm with:  representation an array of real numbers; 100 individuals; crossover $$child = \alpha \cdot (parent1 - parent2) + parent2 ;$$ mutation - reinitialise on a random position the individual's value.

In [1]:


from random import randint, random
from operator import add
from math import cos, pi


def individual(length, vmin, vmax):
    '''
    Create a member of the population - an individual

    length: the number of genes (components)
    vmin: the minimum possible value 
    vmax: the maximum possible value 
    '''
    return [ (random()*(vmax-vmin)+vmin) for x in range(length) ]
    
def population(count, length, vmin, vmax):
    """
    Create a number of individuals (i.e. a population).

    count: the number of individuals in the population
    length: the number of values per individual
    vmin: the minimum possible value 
    vmax: the maximum possible value 
    """
    return [ individual(length, vmin, vmax) for x in range(count) ]

def fitness(individual):
    """
    Determine the fitness of an individual. Lower is better.(min problem)
    For this problem we have the Rastrigin function
    
    individual: the individual to evaluate
    """
    n=len(individual)
    f=0;
    for i in range(n):
        f=f+individual[i]*individual[i]
    return f
    
def mutate(individual, pM, vmin, vmax): 
    '''
    Performs a mutation on an individual with the probability of pM.
    If the event will take place, at a random position a new value will be
    generated in the interval [vmin, vmax]

    individual:the individual to be mutated
    pM: the probability the mutation to occure
    vmin: the minimum possible value 
    vmax: the maximum possible value
    '''
    if pM > random():
            p = randint(0, len(individual)-1)
            individual[p] = random()*(vmax-vmin)+vmin
    return individual
    
def crossover(parent1, parent2):
    '''
    crossover between 2 parents
    '''
    child=[]
    alpha=random()
    for x in range(len(parent1)):
        child.append(alpha*(parent1[x]-parent2[x])+parent2[x])
    return child

def iteration(pop, pM, vmin, vmax):
    '''
    an iteration

    pop: the current population
    pM: the probability the mutation to occure
    vmin: the minimum possible value 
    vmax: the maximum possible value
    '''
    i1=randint(0,len(pop)-1)
    i2=randint(0,len(pop)-1)
    if (i1!=i2):
        c=crossover(pop[i1],pop[i2])
        c=mutate(c, pM, vmin, vmax)
        f1=fitness(pop[i1])
        f2=fitness(pop[i2])
        '''
        the repeated evaluation of the parents can be avoided
        if  next to the values stored in the individuals we 
        keep also their fitnesses 
        '''
        fc=fitness(c)
        if(f1>f2) and (f1>fc):
            pop[i1]=c
        if(f2>f1) and (f2>fc):
            pop[i2]=c
    return pop

def main(noIteratii=10000):
    #PARAMETERS:
    
    #population size
    dimPopulation = 100
    #individual size
    dimIndividual = 2
    #the boundries of the search interval
    vmin = -5.12
    vmax = 5.12
    #the mutation probability
    pM=0.01
    
    P = population(dimPopulation, dimIndividual, vmin, vmax)
    for i in range(noIteratii):
        P = iteration(P, pM, vmin, vmax)

    #print the best individual
    graded = [ (fitness(x), x) for x in P]
    graded =  sorted(graded)
    result=graded[0]
    fitnessOptim=result[0]
    individualOptim=result[1]
    print('Result: The detected minimum point after %d iterations is f(%3.2f %3.2f) = %3.2f'% \
          (noIteratii,individualOptim[0],individualOptim[1], fitnessOptim) )
        
main()

Result: The detected minimum point after 10000 iterations is f(-0.00 0.00) = 0.00


Exercise 1:  Construct a similar algorithm to the one provided as an example for the Bukin function N.6 (search the internet for this function).


In [2]:
# your code here

from random import randint, random
from math import sqrt, fabs


def individual(length, xmin, xmax, ymin, ymax):
    return [(random() * (xmax - xmin) + xmin, random() * (ymax - ymin) + ymin) for _ in range(length)]


def population(count, length, xmin, xmax, ymin, ymax):
    return [individual(length, xmin, xmax, ymin, ymax) for _ in range(count)]


def fitness(individual):
    return sum(
        100 * sqrt(fabs(y - 0.01 * x ** 2)) + 0.01 * fabs(x + 10)
        for x, y in individual
    )


def mutate(individual, pM, xmin, xmax, ymin, ymax):
    if pM > random():
        index = randint(0, len(individual) - 1)
        x, y = individual[index]
        if index == 0:
            x = random() * (xmax - xmin) + xmin
        else:
            y = random() * (ymax - ymin) + ymin
        individual[index] = (x, y)
    return individual


def crossover(parent1, parent2):
    alpha = random()
    child = [
        (
            alpha * (parent1[i][0] - parent2[i][0]) + parent2[i][0],
            alpha * (parent1[i][1] - parent2[i][1]) + parent2[i][1]
        ) for i in range(len(parent1))
    ]
    return child
    

def iteration(pop, pM, xmin, xmax, ymin, ymax):
    new_population = []
    for _ in range(len(pop)):
        parent1_index = randint(0, len(pop) - 1)
        parent2_index = randint(0, len(pop) - 1)
        while parent2_index == parent1_index:  # Ensure distinct parents
            parent2_index = randint(0, len(pop) - 1)
        
        parent1 = pop[parent1_index]
        parent2 = pop[parent2_index]
        
        child = crossover(parent1, parent2)
        child = mutate(child, pM, xmin, xmax, ymin, ymax)
        new_population.append(child)
    
    # Select the best individuals to form the next generation
    combined_population = pop + new_population
    combined_population = sorted([(fitness(x), x) for x in combined_population])
    pop = [individual for _, individual in combined_population[:len(pop)]]
    
    return pop



def main(noIterations=10000):
    dimPopulation = 100
    dimIndividual = 2
    xmin = -15
    xmax = -5
    ymin = -3
    ymax = 3
    pM = 0.08

    P = population(dimPopulation, dimIndividual, xmin, xmax, ymin, ymax)
    for i in range(noIterations):
        P = iteration(P, pM, xmin, xmax, ymin, ymax)

    graded = [(fitness(x), x) for x in P]
    graded = sorted(graded)
    result = graded[0]
    fitnessOptim = result[0]
    individualOptim = result[1]
    print(
        'Result: The detected minimum point after %d iterations is f(%3.2f, %3.2f) = %3.2f'
        % (noIterations, individualOptim[0][0], individualOptim[0][1], fitnessOptim)
    )


main()

Result: The detected minimum point after 10000 iterations is f(-9.38, 0.88) = 0.02


Consider the knapsack problem:

Consider a Knapsack with a total volum equal with $V_{max}$.

There are $n$ objects, with values $(p_i)_{n}$ and volumes $(v_i)_n$.

Solve this problem using a generationist Genetic Algorithm, with a binary representation.

Exercise 2: Initialization
Objective: Implement the initialization step of a genetic algorithm.

In [3]:
import random

def initialize_population(population_size, chromosome_length):
    # generate random a population with population_size number of individuals
    # each individual with the size chromosome_length
    # IN:  population_size, chromosome_length
    # OUT: population
    
    # your code here
    population = []
    for _ in range(population_size):
        individual = [random.randint(0, 1) for _ in range(chromosome_length)]
        population.append(individual)
    return population



# Test the initialization step
population_size = 10
chromosome_length = 8
population = initialize_population(population_size, chromosome_length)
print(population)


[[1, 0, 1, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 0, 1, 1, 0], [0, 1, 0, 1, 0, 1, 0, 1], [0, 0, 1, 0, 1, 0, 1, 0], [0, 1, 0, 1, 1, 0, 1, 1], [0, 1, 0, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0, 1]]


Exercise 3: Fitness Evaluation

Objective: Implement the fitness evaluation step of a genetic algorithm.

In [4]:
def evaluate_fitness(population, values, volumes, Vmax):
    # evaluate the fitness of each individual in the population
    # IN:  population(list): A list of binary strings representing individuals.
    # OUT: fitness_scores (list): Fitness scores for each individual.
    
    # your code here
    fitness_scores = []
    for individual in population:
        total_value = 0
        total_volume = 0
        for i in range(len(individual)):
            if individual[i] == 1:  # If object i is included in the knapsack
                total_value += values[i]
                total_volume += volumes[i]
        # If the total volume of the knapsack exceeds the capacity, penalize the fitness
        if total_volume > Vmax:
            fitness_scores.append(0)  # Penalize by setting fitness to 0
            # Aici la penalizare, e foarte brutal sa pui zero direct
            # Varianta mai blanda:
            # Sum(x[i]c[i])- p* |Vmax - Sum(x[i]v[i])|
        else:
            fitness_scores.append(total_value)  # Fitness is the total value

    return fitness_scores
# Test the fitness evaluation step
values = [10, 5, 8, 3, 12, 6, 4, 2]
volumes = [2, 1, 3, 2, 4, 1, 2, 1]
Vmax = 10
fitness_scores = evaluate_fitness(population, values, volumes, Vmax)
print(fitness_scores)

[32, 4, 0, 21, 16, 24, 26, 16, 18, 25]


Exercise 4: Selection

Objective: Implement the selection step of a genetic algorithm.

In [5]:
def select_parents(population, fitness_scores):
    # select two parents from the population based on the fitness - 
    # the better the fitness, the higher the chance to be selected
    # IN:  population, fitness_scores
    # OUT: selected_parents
     # Calculate the total fitness score of the population
    total_fitness = sum(fitness_scores)
    
    # Calculate the probability of selection for each individual
    selection_probabilities = [fitness / total_fitness for fitness in fitness_scores]
    
    # Perform roulette wheel selection to choose two parents
    selected_parents = []
    for _ in range(2):
        cumulative_probability = 0
        random_value = random.random()
        for i, probability in enumerate(selection_probabilities):
            cumulative_probability += probability
            if random_value <= cumulative_probability:
                selected_parents.append(population[i])
                break
    return selected_parents
    
# Test the selection step
parents = select_parents(population, fitness_scores)
print(parents)

[[0, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 0, 1]]


Exercise 5: Crossover

Objective: Implement the crossover step of a genetic algorithm.

In [6]:
def crossover(parents):
    # create two new offspring by combining the parents
    # IN:  parents
    # OUT: offspring

    # your code here
    parent1, parent2 = parents
    
    # Choose a random crossover point
    crossover_point = random.randint(1, len(parent1) - 1)
    
    # Create the first offspring by combining the genetic material of parent1 and parent2
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    
    # Create the second offspring by combining the genetic material of parent2 and parent1
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
    
    return [offspring1, offspring2]
# Test the crossover step
offspring = crossover(parents)
print(offspring)


[[0, 0, 1, 0, 1, 0, 0, 1], [1, 0, 1, 0, 1, 0, 1, 0]]


Exercise 6: Mutation

Objective: Implement the mutation step of a genetic algorithm.

In [7]:
def mutate(chromosome, mutation_rate):
    # mutate the chromosome by randomly flipping bits
    # IN:  chromosome, mutation_rate
    # OUT: mutated_chromosome

    # your code here
    mutated_chromosome = []
    for bit in chromosome:
        if random.random() < mutation_rate:
            # Flip the bit with a probability equal to the mutation rate
            mutated_bit = 1 - bit
        else:
            mutated_bit = bit
        mutated_chromosome.append(mutated_bit)
    
    return mutated_chromosome

# Test the mutation step
mutation_rate = 0.1
mutated_offspring = [mutate(child, mutation_rate) for child in offspring]
print(mutated_offspring)


[[1, 0, 1, 0, 1, 0, 0, 1], [1, 0, 1, 0, 1, 0, 0, 0]]


Exercise 7: Complete Genetic Algorithm

Objective: Combine all the steps of a genetic algorithm to solve a specific problem.

In [8]:
print(values)
print(volumes)

[10, 5, 8, 3, 12, 6, 4, 2]
[2, 1, 3, 2, 4, 1, 2, 1]


In [9]:
def genetic_algorithm(population_size, chromosome_length, generations, mutation_rate, values, volumes, Vmax):
    # complete genetic algorithm
    # IN:  population_size, chromosome_length, generations, mutation_rate
    # OUT: population

    # initialize the population
    # your code here
    population = initialize_population(population_size, chromosome_length)

    for _ in range(generations):
        # Fitness evaluation
        # your code here
        fitness_scores = evaluate_fitness(population, values, volumes, Vmax)

        # Crossover
        # your code here
        offspring = []
        for i in range(population_size // 2):
            # Selection
            # your code here
            parents = select_parents(population, fitness_scores)
            child1, child2 = crossover(parents)
            # Mutation
            # your code here
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)
            offspring.extend([child1, child2])

        # for i in range(len(offspring)):
        #     offspring[i] = mutate(offspring[i], mutation_rate)

        # Replace the population with the new generation
        # your code here
        population = offspring

    return population

# Test the complete genetic algorithm
population_size = 10
chromosome_length = 8
generations = 100
mutation_rate = 0.1

values = [10, 5, 8, 3, 12, 6, 4, 2]
volumes = [2, 1, 3, 2, 4, 1, 2, 1]
Vmax = 10

final_population = genetic_algorithm(population_size, chromosome_length, generations, mutation_rate, values, volumes, Vmax)
print(final_population)

[[0, 1, 1, 0, 0, 0, 0, 1], [1, 1, 0, 0, 0, 0, 0, 1], [1, 1, 0, 0, 0, 0, 1, 1], [1, 0, 0, 0, 0, 1, 0, 0], [1, 1, 0, 0, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1], [1, 1, 1, 0, 0, 0, 0, 1], [1, 1, 0, 0, 0, 0, 0, 1], [0, 1, 1, 1, 0, 0, 0, 1], [1, 1, 0, 0, 0, 0, 0, 1]]


Exercise 8: Extract the result from the final population

Objective: Get the best individual from the final population.


In [10]:
# determine the best individual from the final population and print it out

# your code here
def extract_best_individual(final_population, values, volumes, Vmax):
    # Evaluate the fitness of each individual in the final population
    fitness_scores = evaluate_fitness(final_population, values, volumes, Vmax)
    # Find the index of the individual with the highest fitness score
    best_index = fitness_scores.index(max(fitness_scores))
    # Retrieve the best individual from the final population
    best_individual = final_population[best_index]
    return best_individual

# Test the extraction of the best individual
best_individual = extract_best_individual(final_population, values, volumes, Vmax)
print("Best individual:", best_individual)


Best individual: [1, 1, 0, 1, 0, 1, 0, 1]
